In [2]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
input_size = (400, 600)  # (h, w)
output_size = (64, 64)  # (h, w)
bbox_gt = np.array([[100, 150, 300, 350]])  # (x1, y1, x2, y2)

# 转换成 特征图 的bbox
bboxes = np.array(bbox_gt[:, :4], dtype=np.float32)
bboxes[:, 0::2] = bbox_gt[:, 0::2] / input_size[1] * output_size[1]
bboxes[:, 1::2] = bbox_gt[:, 1::2] / input_size[0] * output_size[0]
# 防止 bbox 超出范围
bbox = np.array(bboxes[0])
bbox[[0, 2]] = np.clip(bbox[[0, 2]], 0, output_size[1] - 1)
bbox[[1, 3]] = np.clip(bbox[[1, 3]], 0, output_size[0] - 1)
bbox



array([10.666667, 24.      , 32.      , 56.      ], dtype=float32)

In [4]:
def gaussian_radius(bbox_size, min_overlap=0.7):
    """cornernet :以真值点为圆心，点可以移动的的半径，保证移动后的bbox与真值bbox IOU >= min_overlap"""
    height, width = bbox_size

    a1 = 1
    b1 = (height + width)
    c1 = width * height * (1 - min_overlap) / (1 + min_overlap)
    sq1 = np.sqrt(b1 ** 2 - 4 * a1 * c1)
    r1 = (b1 + sq1) / 2

    a2 = 4
    b2 = 2 * (height + width)
    c2 = (1 - min_overlap) * width * height
    sq2 = np.sqrt(b2 ** 2 - 4 * a2 * c2)
    r2 = (b2 + sq2) / 2

    a3 = 4 * min_overlap 
    b3 = -2 * min_overlap * (height + width)
    c3 = (min_overlap - 1) * width * height
    sq3 = np.sqrt(b3 ** 2 - 4 * a3 * c3)
    r3 = (b3 + sq3) / 2
    return min(r1, r2, r3)

In [5]:
h, w = bbox[3] - bbox[1], bbox[2] - bbox[0]
r = gaussian_radius((h, w))
r = max(0, int(r))
ct = np.array([(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2], dtype=np.float32)
ct_int = ct.astype(np.int32)
print(f"{r=}\n{ct=}\n{ct_int=}")

r=7
ct=array([21.333334, 40.      ], dtype=float32)
ct_int=array([21, 40], dtype=int32)


In [31]:
def draw_gaussian(heatmap, center, radius, k=1):
    diameter = 2 * radius + 1
    gaussian = gaussian2D((diameter, diameter), sigma=diameter / 6)

    x, y = int(center[0]), int(center[1])

    height, width = heatmap.shape[0:2]

    left, right = min(x, radius), min(width - x, radius + 1)
    top, bottom = min(y, radius), min(height - y, radius + 1)
    
    masked_heatmap = heatmap[y - top:y + bottom, x - left:x + right]  # 共享内存
    masked_gaussian = gaussian[radius - top:radius + bottom, radius - left:radius + right]
    
    if min(masked_gaussian.shape) > 0 and min(masked_heatmap.shape) > 0:  # TODO debug
        np.maximum(masked_heatmap, masked_gaussian * k, out=masked_heatmap)
        
    return heatmap


def gaussian2D(shape, sigma=1):
    m, n = [(ss - 1.) / 2. for ss in shape]
    y, x = np.ogrid[-m:m + 1, -n:n + 1]

    h = np.exp(-(x * x + y * y) / (2 * sigma * sigma))
    h[h < np.finfo(h.dtype).eps * h.max()] = 0
    return h

In [32]:
hm = np.zeros((output_size[0], output_size[1]), dtype=np.float32)
hm = draw_gaussian(hm, ct_int, r)
hm

gaussian.shape=(15, 15)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)